In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot
import cv2
import os
from os import listdir
import imutils
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [2]:
training_path='C:/Users/Natalie/Desktop/Project/Alzheimer_s Dataset_1/train'
testing_path='C:/Users/Natalie/Desktop/Project/Alzheimer_s Dataset_1/test'

In [3]:
from keras.preprocessing.image import ImageDataGenerator

In [4]:
train_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=training_path, target_size=(224,224),batch_size=20)

test_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=testing_path, target_size=(224,224),batch_size=20)

Found 5121 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


In [5]:
imgs,labels=next(train_batches)

In [6]:
vgg16_model=tf.keras.applications.vgg16.VGG16()

In [7]:
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
from keras.applications import vgg16
type(vgg16_model)

keras.engine.functional.Functional

In [9]:
model=tf.keras.models.Sequential()
for layer in vgg16_model.layers[:-1]:
      model.add(layer)

In [10]:
for layer in model.layers:
      layer.trainable=False

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [12]:
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dense(64,activation='relu'))
model.add(keras.layers.Dense(4,activation='Softmax'))

In [13]:
model.compile(tf.keras.optimizers.SGD(learning_rate=0.0001),loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [14]:
history=model.fit(train_batches,validation_data=test_batches,steps_per_epoch=4,validation_steps=4,epochs=20,verbose=2)

Epoch 1/20
4/4 - 54s - loss: 1.2355 - accuracy: 0.4000 - precision: 0.2500 - recall: 0.0375 - val_loss: 1.1577 - val_accuracy: 0.4250 - val_precision: 0.5000 - val_recall: 0.0125 - 54s/epoch - 13s/step
Epoch 2/20
4/4 - 55s - loss: 1.2802 - accuracy: 0.3000 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.1958 - val_accuracy: 0.3250 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 55s/epoch - 14s/step
Epoch 3/20
4/4 - 55s - loss: 1.1947 - accuracy: 0.3625 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.1321 - val_accuracy: 0.4000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 55s/epoch - 14s/step
Epoch 4/20
4/4 - 60s - loss: 1.1962 - accuracy: 0.4000 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.1434 - val_accuracy: 0.5000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 60s/epoch - 15s/step
Epoch 5/20
4/4 - 67s - loss: 1.1237 - accuracy: 0.4250 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 1.1837 - val_accuracy: 0.4625 - 

In [15]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

train_precision = history.history['precision'][-1]
train_recall = history.history['recall'][-1]
train_f1_score = 2 * (train_precision * train_recall) / (train_precision + train_recall)
    
val_precision = history.history['val_precision'][-1]
val_recall = history.history['val_recall'][-1]
val_f1_score = 2 * (val_precision * val_recall) / (val_precision + val_recall)

In [16]:
best_val_acc = max(history.history['val_accuracy'])
print('Maximum validation accuracy:', best_val_acc)

Maximum validation accuracy: 0.625


In [17]:
best_acc = max(history.history['accuracy'])
print('Maximum training accuracy:', best_acc)

Maximum training accuracy: 0.5375000238418579


In [18]:
print("Training Precision:", train_precision)
print("Training F1 Score:", train_f1_score)
print("Training Recall:",train_recall)

Training Precision: 0.3333333432674408
Training F1 Score: 0.02409638591421102
Training Recall: 0.012500000186264515


In [19]:
print("Testing Precision:", val_precision)
print("Testing Recall:", val_recall)
print("Testing F1 Score:", val_f1_score)

Testing Precision: 0.800000011920929
Testing Recall: 0.05000000074505806
Testing F1 Score: 0.09411764846128576
